
### Import data and Constuct result file

In [ ]:
import pandas as pd
import ast
df = pd.read_csv('./result/ArgKP21+predictions(v2).csv')
df_list = df.values.tolist()
print(df.columns)
print(df_list[0])
kps = df_list[0][3]
kps = ast.literal_eval(kps)
print(kps[0])

In [10]:
# header = ['topic', 'stance', 'argument', 'matched_kp', 'confidence_score']
# data = pd.DataFrame(columns=header)
#
# # Specify the CSV file path
# csv_file_path = './result/KPM_cos_si_bert_embedding.csv'
#
# # Write the DataFrame to the CSV file with the header
# data.to_csv(csv_file_path, index=False)
#
# print(f"Empty CSV file with header created successfully at {csv_file_path}.")

Empty CSV file with header created successfully at ./result/KPM_cos_si_bert_embedding.csv.


In [11]:
# result = pd.read_csv('./result/KPM_cos_si_bert_embedding.csv')
# print(result.columns)

Index(['topic', 'stance', 'argument', 'matched_kp', 'confidence_score'], dtype='object')


In [15]:
# re_topics = []
# re_stances = []
# re_arguments = []
# for line in df_list:
#     topic = line[0]
#     stance = line[1]
#     arguments = ast.literal_eval(line[2])
#     for argument in arguments:
#         re_topics.append(topic)
#         re_stances.append(stance)
#         re_arguments.append(argument)
#
# print(len(re_topics))
# print(len(re_stances))
# print(len(re_arguments))
# result['topic'] = re_topics
# result['stance'] = re_stances
# result['argument'] = re_arguments
# result.to_csv('./result/KPM_cos_si_bert_embedding.csv',index=False)
# print(f"have initialized result file.")

7238
7238
7238
have initialized result file.


### Calculate confidence score and extract the highest one

In [4]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_bert_similarity(sentence1, sentence2):
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize and obtain embeddings for each sentence
    tokens1 = tokenizer(sentence1, return_tensors='pt')
    tokens2 = tokenizer(sentence2, return_tensors='pt')

    with torch.no_grad():
        outputs1 = model(**tokens1)
        outputs2 = model(**tokens2)

    # Extract the embeddings from the model outputs
    embedding1 = outputs1.last_hidden_state.mean(dim=1).numpy()
    embedding2 = outputs2.last_hidden_state.mean(dim=1).numpy()

    # Reshape embeddings for cosine similarity calculation
    embedding1 = np.squeeze(embedding1)
    embedding2 = np.squeeze(embedding2)

    # Calculate cosine similarity
    similarity_score = cosine_similarity([embedding1], [embedding2])[0][0]

    return similarity_score

In [5]:
def find_best_kp(argument, kps):
    highest_score = 0
    best_kp = ''
    for candidate_sentence in kps:
        score = calculate_bert_similarity(argument, candidate_sentence)
        if score > highest_score:
            highest_score = score
            best_kp = candidate_sentence

    return highest_score, best_kp

In [6]:
import pandas as pd
result = pd.read_csv('./result/KPM_cos_si_bert_embedding.csv')
print(result.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(v1)', 'confidence_score(v1)',
       'matched_kp(v2)', 'confidence_score(v2)'],
      dtype='object')


In [7]:
result_list = result.values.tolist()
print(len(result_list))

7238


In [8]:
result_list = result_list[:10]
for index, value in enumerate(result_list):
    row = df.loc[(df['topic'] == value[0]) & df['stance'] == value[1]]
    predicted_kps = ast.literal_eval(row['predict_kps'].iloc[0])
    confidence_score, best_kp = find_best_kp(value[2], predicted_kps)
    result_id = index
    print(result_id)
    print(confidence_score)
    print("best_kp: "+best_kp)
    result.at[result_id,'matched_kp(v2)'] = best_kp
    result.at[result_id,'confidence_score(v2)'] = confidence_score

result.to_csv('./result/KPM_cos_si_bert_embedding.csv', index=False)
print("Add new data successfully!!!!!")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

0
0.807071
best_kp: Assisted suicide could inadvertently put pressure on the terminally ill to end their life out of fear of being a burden to their loved ones.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

1
0.80415773
best_kp: Assisted suicide could potentially open the door for individuals with ulterior motives to coerce or manipulate vulnerable patients into ending their lives.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

...

7
0.8624901
best_kp: Assisted suicide could inadvertently put pressure on the terminally ill to end their life out of fear of being a burden to their loved ones.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

8
0.85559356
best_kp: Assisted suicide contradicts the Hippocratic oath taken by healthcare professionals to preserve life and do no harm.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

9
0.9054855
best_kp: Assisted suicide is essentially aiding in the death of another person, which can be seen as equivalent to murder and should therefore be treated as a criminal offense.
Add new data successfully!!!!!


In [1]:
import pandas as pd

pre = pd.read_csv('./result/KPM_cos_si_bert_embedding.csv')
print(pre.columns)

Index(['topic', 'stance', 'argument', 'matched_kp', 'confidence_score'], dtype='object')


In [2]:
score_list = pre['confidence_score'].values.tolist()
print(sum(score_list)/len(score_list))

0.7812484093844215


### Calculate average confidence score of predition(v2)

In [1]:
import pandas as pd
result = pd.read_csv('./result/KPM_cos_si_bert_embedding.csv')
print(result.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(v1)', 'confidence_score(v1)',
       'matched_kp(v2)', 'confidence_score(v2)', 'matched_kp_gpt4(c_avg)',
       'confidence_score_gpt4(c_avg)', 'matched_kp_gpt4(c_best)',
       'confidence_score_gpt4(c_best)', 'matched_kp_gpt4(c_pca)',
       'confidence_score_gpt4(c_pca)', 'matched_kp(v3)',
       'confidence_score(v3)', 'matched_kp_llama2(c_avg)',
       'confidence_score_llama2(c_avg)', 'matched_kp_llama2(c_best)',
       'confidence_score_llama2(c_best)', 'matched_kp_llama2(c_pca)',
       'confidence_score_llama2(c_pca)', 'matched_kp(v4)',
       'confidence_score(v4)', 'matched_kp(c_pca)', 'confidence_score(c_pca)',
       'matched_kp(c_best)', 'confidence_score(c_best)', 'matched_kp(c_avg)',
       'confidence_score(c_avg)'],
      dtype='object')


In [2]:
confidence_score = result['confidence_score(c_pca)'].values.tolist()
print(sum(confidence_score)/len(confidence_score))

0.7498500729903244


In [3]:
confidence_score = result['confidence_score(c_best)'].values.tolist()
print(sum(confidence_score)/len(confidence_score))

0.7607612517467024


In [6]:
confidence_score = result['confidence_score_llama2(c_pca)'].values.tolist()
print(sum(confidence_score)/len(confidence_score))

0.7819863597974588


In [7]:
confidence_score = result['confidence_score(v3)'].values.tolist()
print(sum(confidence_score)/len(confidence_score))

0.7849173338811833
